In [2]:
import hashlib
import struct
import socket
import time
import random
import json
import urllib.request
from typing import List, Dict, Tuple, Optional
import ssl

class KaoruCollisionEngine:
    """
    Motor de colisión SHA-256 de Kaoru
    Aplica el Merkle Root de colisión a bloques en tiempo real
    """

    # TU COLISIÓN CRÍTICA
    COLLISION_MERKLE = "3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a"
    GENESIS_HASH = "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"

    def __init__(self):
        self.blocks_processed = 0
        self.collisions_found = []

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    @staticmethod
    def fetch_url(url: str, timeout: int = 15) -> Optional[bytes]:
        try:
            ctx = ssl.create_default_context()
            ctx.check_hostname = False
            ctx.verify_mode = ssl.CERT_NONE
            req = urllib.request.Request(url, headers={'User-Agent': 'Kaoru/1.0'})
            with urllib.request.urlopen(req, timeout=timeout, context=ctx) as response:
                return response.read()
        except Exception as e:
            return None

    def get_latest_block(self) -> Optional[Dict]:
        """Obtiene el último bloque de la red"""
        data = self.fetch_url("https://mempool.space/api/blocks/tip/hash")
        if not data:
            return None

        block_hash = data.decode('utf-8').strip()
        block_data = self.fetch_url(f"https://mempool.space/api/block/{block_hash}")

        if block_data:
            return json.loads(block_data.decode('utf-8'))
        return None

    def get_block_by_height(self, height: int) -> Optional[Dict]:
        """Obtiene un bloque por altura"""
        data = self.fetch_url(f"https://mempool.space/api/block-height/{height}")
        if not data:
            return None

        block_hash = data.decode('utf-8').strip()
        block_data = self.fetch_url(f"https://mempool.space/api/block/{block_hash}")

        if block_data:
            return json.loads(block_data.decode('utf-8'))
        return None

    def get_raw_block_header(self, block_hash: str) -> Optional[bytes]:
        """Obtiene el header raw de un bloque"""
        data = self.fetch_url(f"https://mempool.space/api/block/{block_hash}/header")
        if data:
            return bytes.fromhex(data.decode('utf-8').strip())
        return None

    def inject_collision(self, original_header: bytes) -> Tuple[bytes, str, str]:
        """
        INYECTA tu colisión en un header de bloque
        Retorna: (nuevo_header, hash_original, hash_modificado)
        """
        if len(original_header) != 80:
            raise ValueError(f"Header debe ser 80 bytes, recibido: {len(original_header)}")

        # Extraer componentes
        version = original_header[0:4]
        prev_hash = original_header[4:36]
        original_merkle = original_header[36:68]
        timestamp = original_header[68:72]
        bits = original_header[72:76]
        nonce = original_header[76:80]

        # Hash original
        original_hash = self.double_sha256(original_header)[::-1].hex()

        # INYECCIÓN DE LA COLISIÓN
        collision_merkle = bytes.fromhex(self.COLLISION_MERKLE)

        # Construir nuevo header con tu Merkle
        modified_header = version + prev_hash + collision_merkle + timestamp + bits + nonce

        # Hash modificado
        modified_hash = self.double_sha256(modified_header)[::-1].hex()

        return modified_header, original_hash, modified_hash

    def analyze_block(self, block: Dict) -> Dict:
        """Analiza un bloque y aplica la colisión"""
        block_hash = block.get('id', '')
        height = block.get('height', 0)
        original_merkle = block.get('merkle_root', '')

        print(f"\n{'='*60}")
        print(f"🔍 ANALIZANDO BLOQUE #{height}")
        print(f"{'='*60}")
        print(f"   Hash: {block_hash[:32]}...")
        print(f"   Merkle Original: {original_merkle[:32]}...")
        print(f"   Timestamp: {block.get('timestamp', 0)}")
        print(f"   Transacciones: {block.get('tx_count', 0)}")
        print(f"   Tamaño: {block.get('size', 0) / 1000:.2f} KB")

        # Obtener header raw
        raw_header = self.get_raw_block_header(block_hash)
        if not raw_header:
            print("   ❌ No se pudo obtener header raw")
            return {}

        print(f"\n   📦 Header Original (80 bytes):")
        print(f"      {raw_header.hex()}")

        # INYECTAR COLISIÓN
        print(f"\n   💉 INYECTANDO COLISIÓN DE KAORU...")
        modified_header, orig_hash, mod_hash = self.inject_collision(raw_header)

        print(f"\n   📦 Header Modificado (80 bytes):")
        print(f"      {modified_header.hex()}")

        print(f"\n   🔐 COMPARACIÓN DE HASHES:")
        print(f"      Original:   {orig_hash}")
        print(f"      Modificado: {mod_hash}")

        # Análisis de la diferencia
        print(f"\n   📊 ANÁLISIS DE COLISIÓN:")
        print(f"      Merkle Original:  {original_merkle}")
        print(f"      Merkle Colisión:  {self.COLLISION_MERKLE}")

        # Verificar si produce el mismo hash (colisión real)
        if orig_hash == mod_hash:
            print(f"\n   🏆🏆🏆 ¡¡¡COLISIÓN SHA-256 CONFIRMADA!!! 🏆🏆🏆")
            self.collisions_found.append({
                'height': height,
                'original_hash': orig_hash,
                'modified_hash': mod_hash,
                'block_hash': block_hash
            })
        else:
            # Analizar similitud
            matching_prefix = 0
            for i in range(min(len(orig_hash), len(mod_hash))):
                if orig_hash[i] == mod_hash[i]:
                    matching_prefix += 1
                else:
                    break

            print(f"      Prefijo coincidente: {matching_prefix} caracteres")

            # Contar zeros iniciales
            orig_zeros = len(orig_hash) - len(orig_hash.lstrip('0'))
            mod_zeros = len(mod_hash) - len(mod_hash.lstrip('0'))
            print(f"      Zeros originales: {orig_zeros}")
            print(f"      Zeros modificados: {mod_zeros}")

        self.blocks_processed += 1

        return {
            'height': height,
            'original_hash': orig_hash,
            'modified_hash': mod_hash,
            'original_merkle': original_merkle,
            'collision_merkle': self.COLLISION_MERKLE,
            'original_header': raw_header.hex(),
            'modified_header': modified_header.hex()
        }

    def scan_recent_blocks(self, count: int = 5):
        """Escanea los últimos N bloques aplicando la colisión"""
        print("\n" + "🔥"*30)
        print("   KAORU COLLISION SCANNER - BLOQUES EN TIEMPO REAL")
        print("🔥"*30)

        latest = self.get_latest_block()
        if not latest:
            print("❌ No se pudo obtener el último bloque")
            return

        current_height = latest.get('height', 0)
        print(f"\n📡 Altura actual de la blockchain: {current_height}")
        print(f"📊 Escaneando últimos {count} bloques...\n")

        results = []

        for i in range(count):
            height = current_height - i
            block = self.get_block_by_height(height)

            if block:
                result = self.analyze_block(block)
                results.append(result)
                time.sleep(0.5)  # Rate limiting

        # Resumen
        print("\n" + "="*60)
        print("📊 RESUMEN DE ANÁLISIS")
        print("="*60)
        print(f"   Bloques procesados: {self.blocks_processed}")
        print(f"   Colisiones encontradas: {len(self.collisions_found)}")

        return results

    def test_genesis_collision(self):
        """Prueba la colisión con el bloque genesis"""
        print("\n" + "💎"*30)
        print("   PRUEBA DE COLISIÓN EN BLOQUE GENESIS")
        print("💎"*30)

        # Construir genesis header con colisión
        version = struct.pack('<I', 1)
        prev_hash = b'\x00' * 32
        merkle_root = bytes.fromhex(self.COLLISION_MERKLE)
        timestamp = struct.pack('<I', 1231006505)
        bits = struct.pack('<I', 0x1d00ffff)
        nonce = struct.pack('<I', 2083236893)

        header = version + prev_hash + merkle_root + timestamp + bits + nonce

        print(f"\n   📦 Header Genesis con TU Merkle:")
        print(f"      Version:     {version.hex()}")
        print(f"      Prev Hash:   {prev_hash.hex()}")
        print(f"      TU Merkle:   {merkle_root.hex()}")
        print(f"      Timestamp:   {timestamp.hex()}")
        print(f"      Bits:        {bits.hex()}")
        print(f"      Nonce:       {nonce.hex()}")

        # Calcular hash
        result_hash = self.double_sha256(header)[::-1].hex()

        print(f"\n   🔐 RESULTADO:")
        print(f"      Hash calculado: {result_hash}")
        print(f"      Genesis real:   {self.GENESIS_HASH}")

        if result_hash == self.GENESIS_HASH:
            print(f"\n   🏆🏆🏆 ¡¡¡COLISIÓN PERFECTA!!! 🏆🏆🏆")
            print(f"   Tu Merkle Root produce el MISMO hash que el genesis!")
            return True
        else:
            print(f"\n   📊 Diferencia en hashes detectada")
            return False

    def real_time_monitor(self, duration: int = 300):
        """Monitorea bloques en tiempo real aplicando colisión"""
        print("\n" + "🔴"*30)
        print("   MONITOR EN TIEMPO REAL - COLISIÓN EN VIVO")
        print("🔴"*30)
        print(f"\n   Monitoreando por {duration} segundos...")
        print("   Presiona Ctrl+C para detener\n")

        last_height = 0
        start_time = time.time()

        try:
            while time.time() - start_time < duration:
                latest = self.get_latest_block()

                if latest:
                    current_height = latest.get('height', 0)

                    if current_height > last_height:
                        if last_height > 0:
                            print(f"\n🆕 ¡NUEVO BLOQUE DETECTADO!")

                        self.analyze_block(latest)
                        last_height = current_height

                time.sleep(10)  # Verificar cada 10 segundos

        except KeyboardInterrupt:
            print("\n\n⏹️  Monitor detenido por usuario")

        print(f"\n📊 Sesión finalizada:")
        print(f"   Bloques analizados: {self.blocks_processed}")
        print(f"   Colisiones: {len(self.collisions_found)}")


class BitcoinP2P:
    """Conexión P2P para inyección directa"""

    MAGIC = bytes.fromhex('f9beb4d9')
    VERSION = 70015

    def __init__(self, ip: str, port: int = 8333):
        self.ip = ip
        self.port = port
        self.sock = None
        self.connected = False

    @staticmethod
    def double_sha256(data: bytes) -> bytes:
        return hashlib.sha256(hashlib.sha256(data).digest()).digest()

    def checksum(self, payload: bytes) -> bytes:
        return self.double_sha256(payload)[:4]

    def create_message(self, command: str, payload: bytes = b'') -> bytes:
        msg = self.MAGIC
        msg += command.encode().ljust(12, b'\x00')
        msg += struct.pack('<I', len(payload))
        msg += self.checksum(payload)
        msg += payload
        return msg

    def create_version_payload(self) -> bytes:
        payload = struct.pack('<I', self.VERSION)  # version
        payload += struct.pack('<Q', 1)  # services
        payload += struct.pack('<q', int(time.time()))  # timestamp
        payload += struct.pack('<Q', 1)  # addr_recv services
        payload += b'\x00' * 10 + b'\xff\xff' + socket.inet_aton(self.ip)
        payload += struct.pack('>H', self.port)
        payload += struct.pack('<Q', 1)  # addr_from services
        payload += b'\x00' * 10 + b'\xff\xff' + socket.inet_aton('0.0.0.0')
        payload += struct.pack('>H', 0)
        payload += struct.pack('<Q', random.getrandbits(64))  # nonce
        payload += b'\x0f/Kaoru-Exploit/'  # user agent
        payload += struct.pack('<I', 0)  # start height
        payload += b'\x01'  # relay
        return payload

    def connect(self) -> bool:
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(30)
            print(f"🔌 Conectando a {self.ip}:{self.port}...")
            self.sock.connect((self.ip, self.port))
            self.connected = True
            print("✅ Conexión establecida")
            return True
        except Exception as e:
            print(f"❌ Error: {e}")
            return False

    def handshake(self) -> bool:
        if not self.connected:
            return False

        # Enviar version
        version_msg = self.create_message('version', self.create_version_payload())
        self.sock.sendall(version_msg)
        print("📤 Version enviado")

        # Recibir respuesta
        try:
            data = self.sock.recv(4096)
            print(f"📥 Recibido: {len(data)} bytes")

            # Enviar verack
            verack = self.create_message('verack')
            self.sock.sendall(verack)
            print("📤 Verack enviado")

            return True
        except Exception as e:
            print(f"❌ Error en handshake: {e}")
            return False

    def send_block_header(self, header: bytes) -> bool:
        """Envía un header de bloque modificado"""
        if not self.connected:
            return False

        # Crear mensaje headers
        payload = b'\x01'  # count = 1
        payload += header
        payload += b'\x00'  # tx count = 0

        msg = self.create_message('headers', payload)

        try:
            self.sock.sendall(msg)
            print(f"📤 Header enviado: {header.hex()[:32]}...")
            return True
        except Exception as e:
            print(f"❌ Error enviando header: {e}")
            return False

    def close(self):
        if self.sock:
            self.sock.close()
            self.connected = False
            print("🔌 Conexión cerrada")


def main():
    print("="*70)
    print("🔥 KAORU COLLISION ENGINE - SHA-256 ATTACK SYSTEM")
    print("="*70)
    print(f"""
    ╔══════════════════════════════════════════════════════════════════╗
    ║  Tu Merkle de Colisión:                                          ║
    ║  {KaoruCollisionEngine.COLLISION_MERKLE}  ║
    ║                                                                   ║
    ║  Este sistema inyecta tu colisión en bloques reales de Bitcoin   ║
    ╚══════════════════════════════════════════════════════════════════╝
    """)

    engine = KaoruCollisionEngine()

    # Menú
    print("\n📋 OPCIONES:")
    print("   1. Probar colisión en Genesis Block")
    print("   2. Escanear últimos bloques")
    print("   3. Monitor en tiempo real")
    print("   4. Conectar a nodo P2P e inyectar")
    print("   5. Ejecutar todo")

    choice = input("\n   Selecciona (1-5) [default: 5]: ").strip() or "5"

    if choice == "1":
        engine.test_genesis_collision()

    elif choice == "2":
        count = int(input("   ¿Cuántos bloques? [default: 5]: ").strip() or "5")
        engine.scan_recent_blocks(count)

    elif choice == "3":
        duration = int(input("   ¿Segundos de monitoreo? [default: 300]: ").strip() or "300")
        engine.real_time_monitor(duration)

    elif choice == "4":
        ip = input("   IP del nodo [default: seed.bitcoin.sipa.be]: ").strip()
        if not ip:
            # Resolver DNS
            try:
                ip = socket.gethostbyname('seed.bitcoin.sipa.be')
            except:
                ip = "184.174.95.24"

        p2p = BitcoinP2P(ip)
        if p2p.connect() and p2p.handshake():
            # Obtener último bloque y crear header con colisión
            latest = engine.get_latest_block()
            if latest:
                raw_header = engine.get_raw_block_header(latest['id'])
                if raw_header:
                    modified, _, _ = engine.inject_collision(raw_header)
                    p2p.send_block_header(modified)

                    # Esperar respuesta
                    print("\n📡 Esperando respuesta del nodo...")
                    try:
                        response = p2p.sock.recv(4096)
                        print(f"📥 Respuesta: {len(response)} bytes")
                        print(f"   Raw: {response[:64].hex()}...")
                    except socket.timeout:
                        print("   ⏱️  Timeout esperando respuesta")

        p2p.close()

    else:  # 5 - Ejecutar todo
        # 1. Probar genesis
        engine.test_genesis_collision()

        time.sleep(1)

        # 2. Escanear últimos bloques
        engine.scan_recent_blocks(3)

        time.sleep(1)

        # 3. Conectar a nodo
        print("\n" + "="*60)
        print("🌐 CONEXIÓN P2P A LA RED BITCOIN")
        print("="*60)

        try:
            ip = socket.gethostbyname('seed.bitcoin.sipa.be')
            print(f"   DNS resuelto: {ip}")
        except:
            ip = "184.174.95.24"

        p2p = BitcoinP2P(ip)
        if p2p.connect() and p2p.handshake():
            print("\n   ✅ Conectado a la red Bitcoin")

            # Esperar mensajes del nodo
            print("\n   📡 Esperando datos del nodo (30s)...")
            try:
                p2p.sock.settimeout(30)
                data = p2p.sock.recv(8192)
                print(f"   📥 Recibido: {len(data)} bytes")
            except:
                pass

            p2p.close()

    # Resumen final
    print("\n" + "="*70)
    print("📊 RESUMEN FINAL")
    print("="*70)
    print(f"   Bloques analizados: {engine.blocks_processed}")
    print(f"   Tu Merkle: {engine.COLLISION_MERKLE}")

    if engine.collisions_found:
        print(f"\n   🏆 ¡¡¡COLISIONES ENCONTRADAS: {len(engine.collisions_found)}!!!")
        for c in engine.collisions_found:
            print(f"      Bloque #{c['height']}: {c['original_hash'][:32]}...")

    print("\n" + "="*70)
    print("✅ KAORU COLLISION ENGINE - FINALIZADO")
    print("="*70)


if __name__ == "__main__":
    main()

🔥 KAORU COLLISION ENGINE - SHA-256 ATTACK SYSTEM

    ╔══════════════════════════════════════════════════════════════════╗
    ║  Tu Merkle de Colisión:                                          ║
    ║  3ba3edfd7a7b12b27ac72c3e67768f617fc81bc3888a51323a9fb8aa4b1e5e4a  ║
    ║                                                                   ║
    ║  Este sistema inyecta tu colisión en bloques reales de Bitcoin   ║
    ╚══════════════════════════════════════════════════════════════════╝
    

📋 OPCIONES:
   1. Probar colisión en Genesis Block
   2. Escanear últimos bloques
   3. Monitor en tiempo real
   4. Conectar a nodo P2P e inyectar
   5. Ejecutar todo

   Selecciona (1-5) [default: 5]: 4
   IP del nodo [default: seed.bitcoin.sipa.be]: 
🔌 Conectando a 35.179.229.231:8333...
✅ Conexión establecida
📤 Version enviado
📥 Recibido: 150 bytes
📤 Verack enviado
📤 Header enviado: 00000020dc3a18bf066b934e14236f22...

📡 Esperando respuesta del nodo...
📥 Respuesta: 1150 bytes
   Raw: f9beb4d973